In [1]:
!pip install -U sentence-transformers matplotlib seaborn



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Found credentials from IAM Role: cld_d81e97c8fg46h1crfqpx4w37ei-cluster-node-role


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import torch

%matplotlib inline

In [4]:
path="/Users/kyradresen/Downloads/fat clever beach repo anyscale/Utility/0507_mtsamples_final.csv"
df = pd.read_csv(path)
print(df.shape)
df.head()

(2016, 26)


,Unnamed: 0,medical_specialty,transcription,age_related_sentence,extracted_text,word_count,med_masked_transcriptions,pii_masked_transcriptions,synthetic_sentence_nodp_temp0.6_topk50,synthetic_sentence_eps16_temp0.6_topk100,...,synthetic_sentence_nodp_temp1.0_topk100,synthetic_sentence_nodp_temp0.8_topk100,synthetic_sentence_nodp_temp0.6_topk100,synthetic_sentence_eps3_temp0.6_topk100,synthetic_sentence_eps3_temp0.8_topk100,synthetic_sentence_eps3_temp1.2_topk100,synthetic_sentence_eps3_temp1.2_topk50,synthetic_sentence_eps3_temp1_topk50,synthetic_sentence_eps3_temp0.8_topk50,synthetic_sentence_eps3_temp0.6_topk50
0,0,Neurology,"CC:, Confusion and slurred speech.,HX , (prima...","HX , (primarily obtained from boyfriend): This...",(primarily obtained from boyfriend): This 31 y...,43,(primarily obtained from boyfriend): This[AGE]...,(primarily obtained from boyfriend): This[AGE]...,The patient is a 55-year-old Caucasian female ...,The patient is a 38-year-old Caucasian male ag...,...,The patient presents to the patient has been s...,A 16-year-old female who was referred for our ...,The patient was brought to the operating room ...,HISTORY OF PRESENT ILLNESS: The patient is a 3...,"Pulmonary[SEP/H]After a consultation, the pati...",This is the history of multiple minor neurolog...,"According to the patient:The patient, an invas...",Orththroteonthroxy[SEPSEP][SEPES/H]The patient...,Gastroenterology[SEP.]The patient was a 22-yea...,The patient is a man who was treated to a card...
1,1,Cardiovascular/Pulmonary,"PREOPERATIVE DIAGNOSES,Airway obstruction seco...","INDICATIONS FOR SURGERY,The patient is a 50-ye...",The patient is a 50-year-old white male with h...,72,The patient is a[AGE] white[SEX] with history ...,The patient is a[AGE] white[SEX] with history ...,This is a 74-year-old boy who presented to the...,The patient was brought to the hospital with a...,...,The patient received a 13 cm x 4 cm diameter. ...,After informed consent was obtained was brough...,The patient was brought to the operating room ...,The patient is a 13-year-old female who has ha...,The patient who has suffered a serious neck pa...,This is an annual consultation with an adult-s...,.There is an ongoing patient history of bilate...,"Orthopedolosis,PESIAT, a health history for a ...","HISTORY OF PRESENT PRESENT:,The patient is a 2...",The patient is a 37-year-old who is a cardiac-...
2,2,Urology,"PROCEDURE: , Elective male sterilization via b...","PROCEDURE: , Elective male sterilization via b...",Elective male sterilization via bilateral vase...,43,Elective male sterilization via bilateral vase...,Elective male sterilization via bilateral vase...,"HISTORY OF PRESENT ILLNESS:, This is a 51-year...","Cardiovascular/Pulmonary[SEP:],HISTORY OF PRES...",...,The patient's room and brought to the operatin...,The patient was brought to the operating room ...,The patient is a 53-year-old white male with a...,The patient is a 91-year-old male with a histo...,The patient was done with a significant heart ...,The patient is well health at the operating ar...,"From this article, Dr. I have stated that her ...","PANIMA/PES/PROCES,P,This is a patient with the...",The patient is a 25-year-old male who is in a ...,The patient is a 39-year-old man who is diagno...
3,3,Urology,"DESCRIPTION:, The patient was placed in the s...","DESCRIPTION:, The patient was placed in the su...",The patient was placed in the supine position ...,44,The patient was placed in the supine position ...,The patient was placed in the supine position ...,The patient was brought to the operating room ...,The patient with a history of prostate cancer....,...,"The patient tolerated the procedure well.,2, a...",This is a 52-year-old male who fell to have a ...,The patient was brought in the operating room ...,The patient is a 2-year-old male who had under...,:PANOCFETERI: An patient is a 20-year-old woma...,This is a 26-year-old couple who lives for a 2...,The patient is 5. OF OF PRESESTES AND TREASIAT...,;

## Clean up Seperators from Synthetic Data Generation

In [5]:
import pandas as pd
import re

def remove_before_sep(df, start_col):
    """
    Removes everything before '[SEP]' in the specified columns of the DataFrame starting from a given column index.

    Parameters:
    - df: DataFrame containing the data.
    - start_col: The index of the column to start processing from.

    Returns:
    - df: DataFrame with the processed columns.
    """
    # Define the regular expression pattern to match everything before '[SEP]'
    pattern = re.compile(r'.*?\[SEP\]')

    # Apply the regular expression to each column starting from start_col
    for column_name in df.columns[start_col:]:
        df[column_name] = df[column_name].apply(lambda x: re.sub(pattern, '', str(x)))

    return df

In [6]:
df = remove_before_sep(df, 3)

## Calculate Fidelity based on Cosine Similarity

In [13]:
import pandas as pd
import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer

def calculate_fidelity(df, column_original, column_synthetic):
    # Load the SimCSE model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")
    model = AutoModel.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")

    # Extract sentences from the DataFrame columns
    original_sentences = df[column_original].dropna().tolist()
    synthetic_sentences = df[column_synthetic].dropna().tolist()

    # Function to get embeddings for a list of sentences
    def get_embeddings(sentences):
        inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings = outputs.pooler_output  # Use pooler output for SimCSE
        return embeddings

    # Compute embeddings for the original and synthetic sentences
    original_embeddings = get_embeddings(original_sentences)
    synthetic_embeddings = get_embeddings(synthetic_sentences)

    # Calculate pairwise cosine similarity
    similarity_scores = [
        1 - cosine(original_embeddings[i], synthetic_embeddings[i])
        for i in range(len(original_embeddings))
    ]

    # Example aggregation: Calculate the mean of the similarity scores
    mean_similarity = sum(similarity_scores) / len(similarity_scores)

    return mean_similarity


## Medical Data Fidelity Calculation

In [ ]:
# Calculate fidelity for various comparisons
fidelity_extracted_extracted = calculate_fidelity(df, 'extracted_text', 'extracted_text')
fidelity_extracted_pii_masked = calculate_fidelity(df, 'extracted_text', 'pii_masked_transcriptions')
fidelity_extracted_synthetic = calculate_fidelity(df, 'extracted_text', 'ssynthetic_sentence_nodp_temp0.8_topk50')
fidelity_extracted_syntheticeps16 = calculate_fidelity(df, 'extracted_text', 'synthetic_sentence_eps16_temp0.6_topk100')
fidelity_extracted_syntheticeps8 = calculate_fidelity(df, 'extracted_text', 'synthetic_sentence_eps8_temp0.8_topk100')
fidelity_extracted_syntheticeps3 = calculate_fidelity(df, 'extracted_text', 'synthetic_sentence_eps3_temp0.6_topk100')

# Print the results
print(f"Fidelity (Extracted Text vs Extracted Text): {fidelity_extracted_extracted:.3f}")
print(f"Fidelity (Extracted Text vs PII Masked Transcriptions): {fidelity_extracted_pii_masked:.3f}")
print(f"Fidelity (Extracted Text vs Synthetic Sentence No DP): {fidelity_extracted_synthetic:.3f}")
print(f"Fidelity (Extracted Text vs Synthetic Sentence DP eps8): {fidelity_extracted_syntheticeps16:.3f}")
print(f"Fidelity (Extracted Text vs Synthetic Sentence DP eps16): {fidelity_extracted_syntheticeps8:.3f}")
print(f"Fidelity (Extracted Text vs Synthetic Sentence DP No BOS SEP): {fidelity_extracted_syntheticeps3:.3f}")

## SST2 Data Fidelity Calculation

In [ ]:
# Calculate fidelity for the different comparisons
fidelity_sentence_extracted = calculate_fidelity(df, 'sentence', 'sentence')
fidelity_sentence_synthetic_nodp = calculate_fidelity(df, 'sentence', 'synthetic_sentence_nodp')
fidelity_sentence_synthetic_dp_eps16 = calculate_fidelity(df, 'sentence', 'synthetic_sentence_dp_eps16')
fidelity_sentence_synthetic_dp_eps8 = calculate_fidelity(df, 'sentence', 'synthetic_sentence_dp_eps8')
fidelity_sentence_synthetic_dp_eps3 = calculate_fidelity(df, 'sentence', 'synthetic_sentence_dp_eps3')
fidelity_sentence_synthetic_nodp_temp06_topk100 = calculate_fidelity(df, 'sentence', 'synthetic_sentence_nodp_temp0.6_topk100')
fidelity_sentence_synthetic_nodp_temp12_topk100 = calculate_fidelity(df, 'sentence', 'synthetic_sentence_nodp_temp1.2_topk100')
fidelity_sentence_synthetic_nodp_temp08_topk50 = calculate_fidelity(df, 'sentence', 'synthetic_sentence_nodp_temp0.8_topk50')

# Print the results
print(f"Fidelity (Sentence vs Extracted Text): {fidelity_sentence_extracted:.3f}")
print(f"Fidelity (Sentence vs Synthetic Sentence No DP): {fidelity_sentence_synthetic_nodp:.3f}")
print(f"Fidelity (Sentence vs Synthetic Sentence DP Epsilon 16): {fidelity_sentence_synthetic_dp_eps16:.3f}")
print(f"Fidelity (Sentence vs Synthetic Sentence DP Epsilon 8): {fidelity_sentence_synthetic_dp_eps8:.3f}")
print(f"Fidelity (Sentence vs Synthetic Sentence DP Epsilon 3): {fidelity_sentence_synthetic_dp_eps3:.3f}")
print(f"Fidelity (Sentence vs Synthetic Sentence No DP Temp 0.6 Top K 100): {fidelity_sentence_synthetic_nodp_temp06_topk100:.3f}")
print(f"Fidelity (Sentence vs Synthetic Sentence No DP Temp 1.2 Top K 100): {fidelity_sentence_synthetic_nodp_temp12_topk100:.3f}")
print(f"Fidelity (Sentence vs Synthetic Sentence No DP Temp 0.8 Top K 50): {fidelity_sentence_synthetic_nodp_temp08_topk50:.3f}")
